In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
url = "https://raw.githubusercontent.com/jordanchtan/EvaluationData/master/FairyTales/fairy_tales_concat.csv"
# url = "https://raw.githubusercontent.com/jordanchtan/EvaluationData/master/FairyTalesPre/fairy_tales_concat.csv"
df = pd.read_csv(url, encoding='utf8')

sentences = df['message'].values
labels = df['reaction'].values

In [3]:
!pip install transformers

     |████████████████████████████████| 645kB 4.7MB/s 
     |████████████████████████████████| 3.8MB 13.7MB/s 
     |████████████████████████████████| 1.1MB 51.8MB/s 
     |████████████████████████████████| 890kB 52.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a867e6d391d85500317762ed874c98266cb60abf8d23bd856b05e974d2a60ea4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [5]:
import keras.backend as K
import tensorflow as tf

from scipy.spatial.distance import jensenshannon
from numpy import asarray

kl_div = tf.keras.losses.KLDivergence()
 
# calculate the js divergence
def js_divergence(p, q):
	m = 0.5 * (p + q)
	return 0.5 * kl_div(p, m) + 0.5 * kl_div(q, m)

def js_distance(y_true, y_pred):
  return K.sqrt(js_divergence(y_true, y_pred))


Using TensorFlow backend.


In [0]:
from transformers import DistilBertPreTrainedModel, DistilBertModel
import torch.nn as nn
from transformers import BertTokenizer, DistilBertTokenizer


class MyDistilBertForSequenceClassificationReact(DistilBertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.distilbert = DistilBertModel(config)
        self.pre_classifier = nn.Linear(config.dim, config.dim)
        self.classifier = nn.Linear(config.dim, 5)
        # self.classifier = nn.Linear(config.dim, config.num_labels)
        self.dropout = nn.Dropout(config.seq_classif_dropout)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, head_mask=None, inputs_embeds=None, labels=None):
        
        distilbert_output = self.distilbert(
            input_ids=input_ids, attention_mask=attention_mask, head_mask=head_mask, inputs_embeds=inputs_embeds
        )
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        pooled_output = self.pre_classifier(pooled_output)  # (bs, dim)
        pooled_output = nn.ReLU()(pooled_output)  # (bs, dim)
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        logits = self.classifier(pooled_output)  # (bs, dim)
        # new addition jordan
        logits = nn.ReLU()(logits)
        outputs = (logits,) + distilbert_output[1:]
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1, 5), labels.view(-1, 5))
            else:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

class MyDistilBertForSequenceClassificationExpress(DistilBertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.distilbert = DistilBertModel(config)
        self.pre_classifier = nn.Linear(config.dim, config.dim)
        self.classifier = nn.Linear(config.dim, 3)
        # self.classifier = nn.Linear(config.dim, config.num_labels)
        self.dropout = nn.Dropout(config.seq_classif_dropout)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, head_mask=None, inputs_embeds=None, labels=None):
        
        distilbert_output = self.distilbert(
            input_ids=input_ids, attention_mask=attention_mask, head_mask=head_mask, inputs_embeds=inputs_embeds
        )
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        pooled_output = self.pre_classifier(pooled_output)  # (bs, dim)
        pooled_output = nn.ReLU()(pooled_output)  # (bs, dim)
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        logits = self.classifier(pooled_output)  # (bs, dim)
        # new addition jordan
        logits = nn.ReLU()(logits)
        outputs = (logits,) + distilbert_output[1:]
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1, 3), labels.view(-1, 3))
            else:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)



In [0]:
import pandas as pd
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

def predict_sentences(model, tokenizer, sentences):
  max_len = 36

  input_ids = []
  attention_masks = []

  for sent in sentences:
      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = max_len,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      input_ids.append(encoded_dict['input_ids'])
      attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  batch_size = 32
  prediction_data = TensorDataset(input_ids, attention_masks)
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

  # Prediction on test set
  print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))
  # Put model in evaluation mode
  model.eval()
  # Tracking variables 
  predictions = []
  # Predict 
  for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)
    
    b_input_ids, b_input_mask = batch
    
    with torch.no_grad():
        outputs = model(b_input_ids, 
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    predictions.append(logits)
  return predictions

In [0]:
# import pandas as pd
# import numpy as np

# # Load the dataset into a pandas dataframe.
# sentences_holdout = np.load("./drive/My Drive/Colab Notebooks/MainModels/sentences_holdout.npy",allow_pickle=True)
# labels_holdout = np.load("./drive/My Drive/Colab Notebooks/MainModels/labels_holdout.npy",allow_pickle=True)

In [0]:
# print(sentences_holdout.shape)
# print(labels_holdout.shape)

Load express base model

In [0]:
# Load a trained model and vocabulary that you have fine-tuned
output_dir = "./drive/My Drive/Colab Notebooks/MainModels/express_base_model"
model = MyDistilBertForSequenceClassificationExpress.from_pretrained(
    output_dir, # Use the 12-layer BERT model, with an uncased vocab.
    # num_labels = 5, # The number of output labels--2 for binary classification.
    num_labels = 1, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.cuda()
tokenizer = DistilBertTokenizer.from_pretrained(output_dir,do_lower_case=True)


Express base model predictions

In [11]:
express_predictions = predict_sentences(model, tokenizer, sentences)

Predicting labels for 5,108 test sentences...


In [12]:
print(len(express_predictions))
print(express_predictions[0])

160
[[3.1486962 3.024086  3.1031168]
 [3.0001125 3.04738   3.035521 ]
 [3.7224772 3.8547692 3.3431227]
 [2.7693386 3.5925481 3.2458398]
 [2.7586079 3.1759074 2.964889 ]
 [2.7639198 3.216053  3.1495366]
 [2.5587213 3.5013838 3.2401042]
 [3.3141787 3.8761125 3.4742594]
 [3.1055093 2.917144  3.071348 ]
 [2.7009838 3.648583  3.3501458]
 [2.918569  3.058998  3.1661115]
 [2.9796395 2.9538486 3.099547 ]
 [3.6779506 3.8171225 3.407195 ]
 [3.1801808 3.3245637 3.2315383]
 [3.4263954 3.8386765 3.3339663]
 [3.3142595 3.0813804 3.134345 ]
 [3.5757663 3.8822026 3.361874 ]
 [2.7894683 3.9272418 3.3861365]
 [2.953605  2.9339128 2.9852042]
 [2.6343734 3.376688  3.172567 ]
 [2.967202  3.0543604 3.1337838]
 [3.1454592 3.0081577 3.0744352]
 [3.1187432 3.4771683 3.2083728]
 [3.3415735 3.1637552 3.2101471]
 [3.4707997 3.3526049 3.2045567]
 [3.4871783 3.1073399 3.1439657]
 [3.071396  3.1042624 3.1191053]
 [2.8804653 3.1030862 2.9753323]
 [3.2155573 3.6443117 3.3879852]
 [3.3493323 3.2256942 3.1269493]
 [3.57

In [0]:
# np.save('./express_predictions.npy', express_predictions,allow_pickle=True)


Load react base model


In [0]:
# Load a trained model and vocabulary that you have fine-tuned
output_dir = "./drive/My Drive/Colab Notebooks/MainModels/react_base_model"
model = MyDistilBertForSequenceClassificationReact.from_pretrained(
    output_dir, # Use the 12-layer BERT model, with an uncased vocab.
    # num_labels = 5, # The number of output labels--2 for binary classification.
    num_labels = 1, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.cuda()
tokenizer = DistilBertTokenizer.from_pretrained(output_dir,do_lower_case=True)


React base model predictions

In [15]:
react_predictions = predict_sentences(model, tokenizer, sentences)

Predicting labels for 5,108 test sentences...


Combine

In [16]:
import numpy as np
react_predictions = np.vstack(react_predictions)
express_predictions = np.vstack(express_predictions)
print(react_predictions.shape)
print(express_predictions.shape)
data = np.concatenate((react_predictions, express_predictions),axis=1)
print(data.shape)

(5108, 5)
(5108, 3)
(5108, 8)


Meta Learner

In [17]:
from keras.models import load_model
output_dir = "./drive/My Drive/Colab Notebooks/MainModels/"
dependencies = {
    'js_distance': js_distance
}
model = load_model(output_dir + 'ensemble_meta_learner.h5', dependencies)
predictions = model.predict(data)

def map_affect_reaction_to_index(reaction):
  reactions = {"joy": 0, "surprise": 1, "sadness": 2, "anger": 3}
  return reactions[reaction]

def map_fb_idx_2_affect_idx(idx):
  # love_count,wow_count,haha_count,sad_count,angry_count
  if idx == 0 or idx == 2:
    # joy
    return 0
  elif idx == 1:
    # surprise
    return 1
  elif idx == 3:
    # sadness
    return 2
  elif idx == 4:
    # anger
    return 3


labels_idx = list(map(lambda react: map_affect_reaction_to_index(react), labels))
# print(labels_idx[4])
# print(predictions[4])
predictions_argmax = list(map(lambda pred: np.argmax(pred), predictions))
# print(predictions_argmax[4])
predictions_idx = list(map(lambda idx: map_fb_idx_2_affect_idx(idx), predictions_argmax))

names = ["joy", "surprise", "sadness", "anger"]
print(sentences[800])
print(names)
print(labels_idx[800])
print(predictions_idx[800])
from sklearn import metrics

print(metrics.classification_report(labels_idx, predictions_idx, target_names=names))


The music sounded, and the people left the church hand-in-hand, with joy and gladness.
['joy', 'surprise', 'sadness', 'anger']
0
0
              precision    recall  f1-score   support

         joy       0.48      0.87      0.62      1883
    surprise       0.37      0.15      0.22       945
     sadness       0.44      0.56      0.49       988
       anger       0.65      0.03      0.06      1292

    accuracy                           0.47      5108
   macro avg       0.48      0.40      0.35      5108
weighted avg       0.50      0.47      0.38      5108



Manual test

In [0]:
affective_predictions = model.predict(data)